In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import os
jpath = os.path.join

In [2]:
LOSS_DIR = "./pg_binary/log"
writer = SummaryWriter(log_dir=LOSS_DIR)

In [3]:
DATA_DIR = "./spectogram_data_splitted"
MODEL_NAME = "resnet50"
CLASSES = ['lion','frog','cat','chicken','sheep','donkey','monkey','dog','cow','bird']
NUM_CLASSES = len(CLASSES)
BATCH_SIZE = 32
NUM_EPOCHS = 50
FEATURE_EXTRACT = False
INPUT_SIZE = (128, 1024)
LOG_INTERVAL = 300

In [4]:
 # Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(INPUT_SIZE),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
    'test': transforms.Compose([
        transforms.Resize(INPUT_SIZE),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
print("Initializing Datasets and Dataloaders...")
modes = ['train', 'test']
image_datasets = {mode: datasets.ImageFolder(jpath(DATA_DIR, mode), data_transforms[mode]) for mode in ['train', 'test']}
for class_name, idx in image_datasets["train"].class_to_idx.items():
    print(f"Label '{class_name}' convert to id ('{idx}')")
# Create training and validation dataloaders
dataloaders_dict = {mode: torch.utils.data.DataLoader(image_datasets[mode], batch_size=BATCH_SIZE, shuffle=True, num_workers=4) for mode in ['train', 'test']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

Initializing Datasets and Dataloaders...
Label 'bird' convert to id ('0')
Label 'cat' convert to id ('1')
Label 'chicken' convert to id ('2')
Label 'cow' convert to id ('3')
Label 'dog' convert to id ('4')
Label 'donkey' convert to id ('5')
Label 'frog' convert to id ('6')
Label 'lion' convert to id ('7')
Label 'monkey' convert to id ('8')
Label 'sheep' convert to id ('9')
cuda:0


In [5]:
from cnn_finetune import make_model

In [6]:
model = make_model(MODEL_NAME, num_classes=NUM_CLASSES, pretrained=True, input_size=INPUT_SIZE)
model.cuda()
print()

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [8]:
def train(epoch, _iter):
    total_loss = 0
    total_size = 0
    model.train()
    for batch_idx, (data, target) in enumerate(dataloaders_dict["train"]):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        writer.add_scalar('Loss/train', loss.item(), _iter)
        total_loss += loss.item()
        total_size += data.size(0)
        loss.backward()
        optimizer.step()
        if batch_idx % LOG_INTERVAL == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tAverage loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(dataloaders_dict["train"].dataset),
                100. * batch_idx / len(dataloaders_dict["train"]), total_loss / total_size))
        _iter += 1
    return _iter

def val(_iter):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in dataloaders_dict["test"]:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).long().cpu().sum().item()

    test_loss /= len(dataloaders_dict["test"].dataset)
    writer.add_scalar('Loss/val', test_loss, _iter)
    writer.add_scalar('Accuracy/val', correct / len(dataloaders_dict["test"].dataset), _iter)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(dataloaders_dict["test"].dataset),
        100. * correct / len(dataloaders_dict["test"].dataset)))
    
def eval():
    model.eval()
    test_loss = 0
    correct = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for data, target in dataloaders_dict["test"]:
            y_true += target.data.tolist()
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.data.max(1, keepdim=True)[1]
            y_pred += pred.squeeze(1).tolist()
            correct += pred.eq(target.data.view_as(pred)).long().cpu().sum().item()
    test_loss /= len(dataloaders_dict["test"].dataset)
    writer.add_scalar('Accuracy/val', correct / len(dataloaders_dict["test"].dataset), _iter)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(dataloaders_dict["test"].dataset),
        100. * correct / len(dataloaders_dict["test"].dataset)))
    target_names = [0]*NUM_CLASSES
    correct = [i for i, j in zip(y_pred, y_true) if i == j]
    for class_name, idx in image_datasets["test"].class_to_idx.items():
        target_names[idx] = class_name
#         print(f"Class/{class_name}", correct.count(idx), y_true.count(idx))
        writer.add_scalar(f"Class/{class_name}", correct.count(idx) / y_true.count(idx), _iter)
    print(target_names)
    print(classification_report(y_true, y_pred, target_names=target_names))

In [9]:
_iter = 1
for epoch in range(1, NUM_EPOCHS + 1):
    _iter = train(epoch, _iter)

Train Epoch: 1 [0/691 (0%)]	Average loss: 0.073546
Train Epoch: 2 [0/691 (0%)]	Average loss: 0.051914
Train Epoch: 3 [0/691 (0%)]	Average loss: 0.035629
Train Epoch: 4 [0/691 (0%)]	Average loss: 0.025949
Train Epoch: 5 [0/691 (0%)]	Average loss: 0.024380
Train Epoch: 6 [0/691 (0%)]	Average loss: 0.010521
Train Epoch: 7 [0/691 (0%)]	Average loss: 0.008859
Train Epoch: 8 [0/691 (0%)]	Average loss: 0.004936
Train Epoch: 9 [0/691 (0%)]	Average loss: 0.006960
Train Epoch: 10 [0/691 (0%)]	Average loss: 0.005105
Train Epoch: 11 [0/691 (0%)]	Average loss: 0.003456
Train Epoch: 12 [0/691 (0%)]	Average loss: 0.001597
Train Epoch: 13 [0/691 (0%)]	Average loss: 0.001724
Train Epoch: 14 [0/691 (0%)]	Average loss: 0.001067
Train Epoch: 15 [0/691 (0%)]	Average loss: 0.001104
Train Epoch: 16 [0/691 (0%)]	Average loss: 0.000732
Train Epoch: 17 [0/691 (0%)]	Average loss: 0.000316
Train Epoch: 18 [0/691 (0%)]	Average loss: 0.000800
Train Epoch: 19 [0/691 (0%)]	Average loss: 0.000580
Train Epoch: 20 [0/69

In [10]:
eval()


Test set: Average loss: 0.0156, Accuracy: 156/174 (90%)

['bird', 'cat', 'chicken', 'cow', 'dog', 'donkey', 'frog', 'lion', 'monkey', 'sheep']
              precision    recall  f1-score   support

        bird       0.98      1.00      0.99        40
         cat       0.90      0.95      0.93        40
     chicken       0.67      0.67      0.67         6
         cow       0.93      0.93      0.93        15
         dog       0.92      0.87      0.89        39
      donkey       0.83      1.00      0.91         5
        frog       1.00      0.71      0.83         7
        lion       0.57      0.89      0.70         9
      monkey       1.00      0.80      0.89         5
       sheep       1.00      0.50      0.67         8

    accuracy                           0.90       174
   macro avg       0.88      0.83      0.84       174
weighted avg       0.91      0.90      0.90       174



In [11]:
from datetime import date

In [12]:
date = date.today().strftime("%Y_%m_%d")
if not os.path.exists('./models'):
  os.makedirs('./models')
torch.save({
            'epoch': NUM_EPOCHS,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            }, f"./models/model_{date}.pth")